# How to Autoannotate images in YOLO format, using DataTools class
---
Tools for Semi-supervised  auto-annotation of images.

###Pre-work
---
Check if running GPU (which of course speeds things a lot).

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


###Clone and install BaseballCV repo

In [2]:
!git clone https://github.com/dylandru/BaseballCV.git
%cd BaseballCV
!pip install -r requirements.txt


Cloning into 'BaseballCV'...
remote: Enumerating objects: 830, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 830 (delta 97), reused 243 (delta 85), pack-reused 567 (from 1)
Receiving objects: 100% (830/830), 360.74 MiB | 24.44 MiB/s, done.
Resolving deltas: 100% (326/326), done.
Updating files: 100% (153/153), done.
/content/BaseballCV
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:

###Import needed libraries
---
The DataTools class provides all the pipeline needed for fetching videos (from Savant), to splitting them in frames, to auto-annotate using your prefered model from those available in the repo).

In [3]:
from ultralytics import YOLO
from scripts.dataset_tools import DataTools

data_tools = DataTools()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


###Define the parameters for video searching and extraction

In [ ]:
output_frames_folder = "cv_dataset"      #You can change this option
video_download_folder = "raw_videos"     #You can change this option
max_plays = 10                           #You can change this option
max_num_frames = 100                     #You can change this option
max_videos_per_game = 10                 #You can change this option
start_date = "2024-05-22"                #You can change this option
end_date = "2024-05-25"                  #You can change this option
delete_savant_videos = True              #You can change this option


start_date = datetime.strptime(start_date, "%Y-%m-%d")
end_date = datetime.strptime(end_date, "%Y-%m-%d")


###Run the dataset generator

In [5]:
data_tools.generate_photo_dataset(
                           output_frames_folder=output_frames_folder,
                           video_download_folder=video_download_folder,
                           max_plays=max_plays,
                           max_num_frames=max_num_frames,
                           max_videos_per_game=max_videos_per_game,
                           start_date=start_date,
                           end_date=end_date,
                           delete_savant_videos=delete_savant_videos)

This is a large query, it may take a moment to complete


100%|██████████| 4/4 [00:07<00:00,  1.78s/it]


Video downloaded to raw_videos/747199_e73d5018-fa51-465b-b078-629fa56c3a10.mp4
Video downloaded to raw_videos/747199_224d7518-a239-4cc1-bec7-0fa8cadfd060.mp4
Video downloaded to raw_videos/747199_9abffb0f-0ce0-4ede-be21-55ec22f751ad.mp4
Video downloaded to raw_videos/747199_f584e3f8-3db6-4516-8c64-1e442fbb3554.mp4
Video downloaded to raw_videos/747199_e648ccb4-7f62-4fff-a400-da49456109b5.mp4
Video downloaded to raw_videos/747199_9cff386e-87e2-42d1-87ea-ae74943532f0.mp4
Video downloaded to raw_videos/747199_6c466225-5de6-451b-9f42-c0d60c3d6d08.mp4
Video downloaded to raw_videos/747199_e3cbcec5-5492-4931-947c-b5c0f25ce444.mp4
Video downloaded to raw_videos/747199_3d82a54e-b234-4abe-8665-e222180e7c2a.mp4
Video downloaded to raw_videos/747199_de5cb132-a05d-4d10-baa9-60bd10b9bf9e.mp4
Extracted 100 frames from 10 videos over 1 games.
Deleted raw_videos


###Define the parameters for model election, mode, confidence and device.

*Remember to change 'cpu' to 'cuda' if you are using a CUDA device.*

In [6]:
model_alias="glove_tracking"
model_type='detection'
image_dir="cv_dataset"
output_dir="labeled_dataset"
conf=.50
device='cpu'

###Run the auto-annotation process

In [7]:
data_tools.automated_annotation(
                             model_alias=model_alias,
                             model_type=model_type,
                             image_dir=image_dir,
                             output_dir=output_dir,
                             conf=conf,
                             device=device)

Model downloaded to models/glove_tracking/model_weights/glove_tracking.pt


Annotating images: 100%|██████████| 100/100 [04:34<00:00,  2.75s/it]

Annotation process complete.


##Manual part of the process
---
Depending on the level of confidence selected, and actually for most cases, you will need to check the results and filter out images with wrong/missing annotations.

The following viewer can help identifying some of those images, but it's slow. A local or any other type of faster implementation would be more helpful.

In [8]:
import cv2
import os
import glob
import numpy as np
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
from IPython.display import display
from scripts.load_tools import LoadTools

load_tools = LoadTools()

# Example class names (replace with your actual class names)
model = YOLO(load_tools.load_model(model_alias))
dict_classes = model.names

# Load annotations and images (assuming annotations are stored in YOLO format)
def load_annotations(annotation_folder, image_folder):
    annotations = {}
    image_paths = sorted(glob.glob(os.path.join(image_folder, "*.jpg")))  # Adjust for your image format
    for image_path in image_paths:
        image_name = os.path.basename(image_path).replace(".jpg", "")
        annotation_file = os.path.join(annotation_folder, f"{image_name}.txt")
        if os.path.exists(annotation_file):
            with open(annotation_file, 'r') as f:
                bboxes = [list(map(float, line.strip().split())) for line in f.readlines()]
                annotations[image_name] = bboxes
    return image_paths, annotations

# Draw bounding boxes on the image
def draw_bboxes(image, bboxes):
    h, w, _ = image.shape
    for bbox in bboxes:
        class_id, x_center, y_center, width, height = bbox

        # Convert YOLO format to pixel coordinates
        x_min = int((x_center - width / 2) * w)
        y_min = int((y_center - height / 2) * h)
        x_max = int((x_center + width / 2) * w)
        y_max = int((y_center + height / 2) * h)

        # Draw the bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        # Put the class name as label
        class_name = dict_classes.get(int(class_id), "Unknown")
        cv2.putText(image, class_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return image

# Folder paths (update with your paths)
annotation_folder = f'{output_dir}/annotations/'  # YOLO annotations
image_folder = f'{output_dir}/'  # Image folder

# Load annotations and images
image_paths, annotations = load_annotations(annotation_folder, image_folder)

# Initialize viewer
idx = 0
total_images = len(image_paths)

# Function to display the current image with annotations and title
def show_image(idx):
    image_path = image_paths[idx]
    image_name = os.path.basename(image_path).replace(".jpg", "")

    # Load image
    image = cv2.imread(image_path)

    # Draw bounding boxes if annotations exist
    if image_name in annotations:
        image = draw_bboxes(image, annotations[image_name])

    # Display image name
    print(f"Displaying: {image_name}")

    # Display image
    cv2_imshow(image)

# Widget for image navigation
button_back = widgets.Button(description="Previous")
button_next = widgets.Button(description="Next")
image_display = widgets.Output()

# Button callback functions
def on_back_button_clicked(b):
    global idx
    idx = (idx - 1) % total_images
    update_image()

def on_next_button_clicked(b):
    global idx
    idx = (idx + 1) % total_images
    update_image()

def update_image():
    with image_display:
        image_display.clear_output(wait=True)
        show_image(idx)

# Link buttons to functions
button_back.on_click(on_back_button_clicked)
button_next.on_click(on_next_button_clicked)

# Display the widgets and the first image
display(widgets.HBox([button_back, button_next]), image_display)
update_image()


Model found at models/glove_tracking/model_weights/glove_tracking.pt


Output()